<a href="https://colab.research.google.com/github/Tharindakarawita/Simple-python-Sripts-for-Galaxy/blob/main/ncbi_gff_file_to_gene_fasta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fusion Gene Annotation Extraction Script

## Overview

This script extracts "fusion" annotated gene names from a GFF file, then retrieves the corresponding Gene IDs. It subsequently searches through the amino acid sequences to identify the corresponding amino acid sequence and outputs this information. The script is designed for use within the Galaxy platform, where users can easily run the script with their GFF and amino acid sequence files.

---

## Features

- **Extracts "fusion" annotated gene names** from a GFF file.
- **Retrieves Gene IDs** associated with the "fusion" annotations.
- **Searches amino acid sequences** to find corresponding protein sequences.
- **Outputs results** in a readable format, including matched amino acid sequences.

---

## Requirements

- **Python 3.x**: Ensure Python 3 is installed.
- **Required libraries**:
  - `BioPython` (for handling sequence data)
  - `pandas` (for processing the GFF file)
  - `Galaxy` (for integrating the script with the Galaxy platform)

---

## Input

1. **GFF File**: A GFF file containing gene annotations, specifically fusion gene annotations.
2. **Amino Acid Sequence File**: A file containing the corresponding amino acid sequences (e.g., FASTA format).

---

## Output

- A file containing the **Gene ID** and **corresponding amino acid sequence** for the fusion genes.

---

## Script Usage

### Step 1: Prepare Your Files

1. **GFF File**: Ensure that your GFF file contains annotations of the "fusion" genes.
2. **Amino Acid Sequences**: Prepare a FASTA file with amino acid sequences corresponding to the genes in your GFF file.

### Step 2: Upload Files to Galaxy

- Log into Galaxy.
- Upload both the GFF file and the amino acid sequence file.

### Step 3: Running the Script

1. Use the Galaxy interface to find and select the script.
2. Provide the following inputs:
   - GFF file (Fusion gene annotations).
   - Amino acid sequence file (FASTA format).

### Step 4: Execute and Retrieve Results

- Once the script is executed, the output file will display:
  - The **Gene ID** for each fusion gene.
  - The **corresponding amino acid sequence** for each Gene ID.

---

## Example Command

Here’s an example of how the script works from the command line (if you prefer to run it outside Galaxy):

```bash
python fusion_gene_extraction.py --gff fusion_annotations.gff --aa_sequence amino_acids.fasta --output fusion_gene_results.txt
```

### Parameters:
- `--gff`: Path to the GFF file with fusion gene annotations.
- `--aa_sequence`: Path to the FASTA file containing amino acid sequences.
- `--output`: Path to the file where results will be saved.

---

## Troubleshooting

- **Missing Gene IDs**: If no Gene IDs are extracted, ensure that the GFF file correctly includes the `gene_id` attribute in the `Attributes` column.
- **No Sequence Match**: If a gene ID is missing from the amino acid sequence file, check if the sequence is named correctly and matches the Gene IDs from the GFF file.

---

## License

This script is distributed under the MIT License. See `LICENSE` for more information.

---

## Contributing

Feel free to fork this repository and contribute by submitting pull requests. If you encounter any issues, open an issue in the GitHub repository.

---

## Contact

For questions or feedback, please contact [Your Name/Email].



In [ ]:
import csv
from Bio import SeqIO
import os

# List of GFF/FASTA files to process
gff_fasta_files = ['data/1_KU821121.1.gff3', 'data/3_PP947686.1.gff3', 'data/2_NC_039199.1.gff3']  # Add your file paths

# Function to process each GFF/FASTA file
def process_file(gff_fasta_file):
    # Initialize list to store gene IDs
    genes = []

    # Step 1: Extract gene IDs where "fusion" is in the product attribute (from the GFF file)
    with open(gff_fasta_file, 'r') as file:
        reader = csv.reader(file, delimiter='\t')

        # Loop through each line in the file
        for row in reader:
            if not row or row[0].startswith("#"):  # Skip empty or comment lines
                continue

            # Ensure the row has enough columns (GFF should have 9 columns)
            if len(row) < 9:
                continue  # Skip malformed lines

            # The Attributes column (index 8) contains the information we want to extract
            attributes = row[8]

            # Check if the word 'fusion' is present in the product attribute
            if 'fusion' in attributes:
                # Extract the ID value
                id_value = None
                product_value = None

                # Split the attributes to find ID and product
                for attr in attributes.split(';'):
                    if attr.startswith('ID='):
                        id_value = attr.split('=')[1]
                    elif attr.startswith('product='):
                        product_value = attr.split('=')[1]

                # Append the ID if 'fusion' is found in the product value
                if id_value and product_value:
                    genes.append(id_value)

    # Step 2: Parse the FASTA file and extract protein sequences based on gene IDs
    fasta_sequences = SeqIO.to_dict(SeqIO.parse(gff_fasta_file, "fasta"))
    protein_sequences = []

    # Read and process the GFF file again to extract sequences for matching gene IDs
    with open(gff_fasta_file) as handle:
        for line in handle:
            if line.startswith("#"):
                continue  # Skip comment lines
            fields = line.strip().split('\t')

            # Ensure the line has enough columns (GFF should have 9 columns)
            if len(fields) < 9:
                continue  # Skip malformed lines

            seq_id = fields[0]
            feature_type = fields[2]

            # Extract name or ID from attributes column (9th column)
            attributes = fields[8]
            attributes_dict = {k: v for k, v in [x.split('=') for x in attributes.split(';')]}

            # Extract the name or ID (use "ID" or "Name", whichever is available)
            feature_name = attributes_dict.get('ID', attributes_dict.get('Name', 'Unknown'))

            # Check for CDS features in the GFF3 and matching gene IDs
            if feature_type == 'CDS' and feature_name in genes:
                start = int(fields[3]) - 1  # Convert to zero-based index
                end = int(fields[4])
                strand = fields[6]

                # Extract sequence from FASTA
                sequence = fasta_sequences[seq_id].seq[start:end]

                # If on negative strand, reverse complement the sequence
                if strand == "-":
                    sequence = sequence.reverse_complement()

                # Translate nucleotide sequence to protein (amino acid) sequence
                protein_sequence = sequence.translate()

                # Store the protein sequence along with its name
                protein_sequences.append((feature_name, protein_sequence))

    # Step 3: Save the selected protein sequences as FASTA files
    base_filename = os.path.basename(gff_fasta_file).split('.')[0]  # Extract the base file name (without extension)
    for name, protein in protein_sequences:
        # Combine base filename and gene name for output file
        output_file = f"{base_filename}_{name}_AA_seq.fasta"

        with open(output_file, "w") as output_handle:
            output_handle.write(f">{name}\n")
            output_handle.write(str(protein) + "\n")

        print(f"Saved protein sequence for {name} to {output_file}")

# Loop through each GFF/FASTA file in the list and process them
for gff_fasta_file in gff_fasta_files:
    print(f"Processing file: {gff_fasta_file}")
    process_file(gff_fasta_file)


In [ ]:
put("1_KU821121_F.p01_AA_seq.fasta")
put("2_NC_039199_D1Y22_gp4.p01_AA_seq.fasta")
put("3_PP947686_XCG46078.1_AA_seq.fasta")